In [4]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import os
import json

> # 데이터 가져오기

이번 대회에서 제공하는 정보는 멜론 서비스에서 수집된 플레이리스트의 일부분입니다. 대회 설명에서 언급한것과 같이, 곡에 대한 Mel-spectrogram 데이터는 용량이 방대합니다. 베이스라인 코드에서와 같이, 해당 데이터를 사용하지 않고도 문제를 해결하는 방법론이 있으니, 컴퓨터의 용량과 컴퓨팅 파워가 충분할 때 사용하세요.

### song_meta.json: 곡 메타데이터
총 707,989개의 곡에 대한 메타데이터가 수록되어 있습니다.  
개발 데이터와 평가 데이터에 수록된 모든 곡에 대한 메타데이터가 포함되어 있습니다.  
필드 설명  
_id: 곡 ID  
album_id: 앨범 ID  
artist_id_basket: 아티스트 ID 리스트  
artist_name_basket: 아티스트 리스트  
song_name: 곡 제목  
song_gn_gnr_basket: 곡 장르 리스트  
song_gn_dtl_gnr_basket: 곡 세부 장르 리스트  
issue_date: 발매일  
메타데이터의 모든 정보는 저작권자의 비공개 여부 전환, 곡 삭제, 메타데이터 수정 등으로 유효하지 않거나 변동될 수 있습니다.  
genre_gn_all.json:  
곡 메타데이터에 수록된 장르에 대한 정보입니다. 위 song_meta.json 에서 song_gn_gnr_basket 과 song_gn_dtl_gnr_basket 에 들어가는 정보들에 대한 메타데이터입니다.  

### train.json:  
모델 학습용 파일로, 115,071개 플레이리스트의 원본 데이터가 수록되어 있습니다.  
필드 설명  
id: 플레이리스트 ID  
plylst_title: 플레이리스트 제목  
tags: 태그 리스트  
songs: 곡 리스트  
like_cnt: 좋아요 개수  
updt_date: 수정 날짜  
### val.json:

공개 리더보드용 문제 파일로, 23,015개 플레이리스트에 대한 문제가 수록되어 있습니다. 모든 데이터가 수록되어있는 train 파일과는 다르게, 곡과 태그의 일부가 수록되어 있습니다.  
필드 설명  
id: 플레이리스트 ID  
plylst_title: 플레이리스트 제목  
tags: 태그 리스트  
songs: 곡 리스트  
like_cnt: 좋아요 개수  
updt_date: 수정 날짜  
### test.json:

파이널 리더보드용 문제 파일로, 10,740개 플레이리스트에 대한 문제가 수록되어 있습니다. 모든 데이터가 수록되어있는 train 파일과는 다르게, 곡과 태그의 일부가 수록되어 있습니다.  
필드 설명  
id: 플레이리스트 ID  
plylst_title: 플레이리스트 제목  
tags: 태그 리스트  
songs: 곡 리스트  
like_cnt: 좋아요 개수  
updt_date: 수정 날짜  


> ### 장르

In [18]:
data_root = '/opt/ml/finalproject/kakaomusic'
genre_address = os.path.join(data_root,'genre_gn_all.json')
meta_address = os.path.join(data_root,'song_meta.json')
train_address = os.path.join(data_root,'train.json')



In [10]:
with open(genre_address) as json_file:
    genre_dic = json.load(json_file)
print(genre_dic.values())

dict_values(['발라드', '세부장르전체', "'80", "'90", "'00", "'10-", '댄스', '세부장르전체', "'80", "'90", "'00", "'10-", '랩/힙합', '세부장르전체', '랩 스타일', '보컬 스타일', '언더그라운드 힙합', '시대별', 'R&B/Soul', '세부장르전체', '어반', 'R&B', '인디음악', '세부장르전체', '포크', '록', '일렉', '힙합', '발라드', "'90", "'00", "'10-", '록/메탈', '세부장르전체', "'70", "'80", "'90", "'00", "'10-", '성인가요', '세부장르전체', '신세대트로트', '전설의트로트', '뽕짝트로트', '트로트메들리', "트로트'60-'70", "트로트'80-'90", "트로트'00-", "성인가요'80-'90", "성인가요'00-", '포크/블루스', '세부장르전체', "'60-'70", "'80-'90", "'00", "'10-", 'POP', '세부장르전체', '얼터너티브팝', '올디스', '월드팝', "'60-'70", "'80-'90", "'00", "'10-", '록/메탈', '세부장르전체', '모던록', '얼터너티브록', '프로그레시브/아트록', '하드록', '헤비메탈', '뉴메탈/하드코어', '포스트록', "'60", "'70", "'80", "'90", "'00", "'10-", '일렉트로니카', '세부장르전체', '일렉트로니카', '하우스', '클럽뮤직', '일렉트로닉팝', "'80", "'90", "'00", "'10-", '랩/힙합', '세부장르전체', '팝랩', '얼터너티브힙합', '갱스터/하드코어랩', 'East&West', "'80", "'90", "'00", "'10-", 'R&B/Soul', '세부장르전체', '컨템포러리 R&B', '소울', '어반', "'60-'70", "'80-'90", "'00", "'10-", '포크/블루스/컨트리', '세부장르전체', '포크', '블루스', 

In [16]:
len(genre_dic)

254

> ## 노래 메타데이터

In [ ]:
with open(meta_address) as json_file:
    meta_dict = json.load(json_file)

In [14]:
print(meta_dict[0])
print(meta_dict[4])



{'song_gn_dtl_gnr_basket': ['GN0901'], 'issue_date': '20140512', 'album_name': '불후의 명곡 - 7080 추억의 얄개시대 팝송베스트', 'album_id': 2255639, 'artist_id_basket': [2727], 'song_name': 'Feelings', 'song_gn_gnr_basket': ['GN0900'], 'artist_name_basket': ['Various Artists'], 'id': 0}
{'song_gn_dtl_gnr_basket': ['GN1802', 'GN1801'], 'issue_date': '20110824', 'album_id': 2008470, 'artist_id_basket': [560160], 'song_name': '그남자 그여자', 'song_gn_gnr_basket': ['GN1800'], 'album_name': '그남자 그여자', 'artist_name_basket': ['Jude Law'], 'id': 4}


In [15]:
print(genre_dic['GN0901'])
print(genre_dic['GN0900'])
print(genre_dic['GN1802'])
print(genre_dic['GN1801'])
print(genre_dic['GN1800'])




세부장르전체
POP
이지리스닝
세부장르전체
뉴에이지


In [17]:
len(meta_dict)

707989

> # Train data

In [19]:
with open(train_address) as json_file:
    train_dict = json.load(json_file)

In [21]:
train_dict[25]

{'tags': ['잔잔한', '밤', '새벽'],
 'id': 6143,
 'plylst_title': '새벽 찬바람이 침대위를 감쌀때..(국외)',
 'songs': [355345,
  40937,
  172532,
  369114,
  486265,
  35945,
  385005,
  453038,
  341760,
  239059,
  397008,
  304937,
  326424,
  361284,
  516797,
  400781,
  347388,
  295050,
  170021,
  288141,
  255838,
  406349,
  425398,
  693110,
  183446,
  615796,
  174644,
  317212,
  317069,
  271210,
  173065,
  102950,
  636579,
  406012,
  223134,
  134760,
  572928,
  419015,
  306385,
  171884,
  240201,
  16320,
  52283,
  273837,
  565181],
 'like_cnt': 15,
 'updt_date': '2016-05-10 00:29:03.000'}

In [22]:
len(train_dict)

115071

> # train 속한 노래들에 태그들을 입히기
> ### TODO : set 형식으로 바꾸기

In [24]:
sampled_train = train_dict[:100]


In [28]:
train_song2tags = {}
train_song2tags['1'] = ['락']
if('1' in train_song2tags):
    temp = train_song2tags['1']
    temp.append('사랑')
    train_song2tags['1'] = temp
print(train_song2tags)


{'1': ['락', '사랑']}


In [30]:
a = []
a.append('love')
print(a)

['love']


In [51]:
train_song2tags = {}
for one_playll in train_dict:
    # print(one_playll)
    songs = one_playll['songs']
    tags = one_playll['tags']
    plylst_title = one_playll['plylst_title']
    for song in songs:
        # 처음 보는 노래이면
        if(song not in train_song2tags):
            empty_ll = []
            empty_ll.append(plylst_title)
            for tag in tags:
                empty_ll.append(tag)
            train_song2tags[song] = empty_ll
        # 이미 노래가 저장되어있을때는 태그를 추가하쟈
        elif(song in train_song2tags):
            from_ll = train_song2tags[song]
            from_ll
            from_ll.append(plylst_title)
            for tag in tags:
                from_ll.append(tag)
            train_song2tags[song] = from_ll
    # print(train_song2tags)
    # break
# temp = train_song2tags[:2]
# print(temp)








In [52]:
song_id_ll = []
tag_ll = []

for song in train_song2tags.items():
    song_id = song[0]
    tag = song[1]
    song_id_ll.append(song_id)
    tag_ll.append(tag)

In [53]:
song_id_ll
tag_ll

song2tag_df = pd.DataFrame(
    data={
        'songID' : song_id_ll,
        'tag' : tag_ll
    }
)


In [54]:
song2tag_df.sample(5)

,songID,tag
24084,245872,"[클럽 일렉트로닉, 클럽, 매장음악, ready for 서울재즈페스티벌 2019 s..."
304507,672132,"[알려졌으면 하는 인디 곡들(좋아요 100개↓), 어쿠스틱, 인디, 숨겨진, 지금 ..."
510332,511327,"[독보적인 음색 밤 분위기를 가진 여성보컬 R&B, 감성, 밤, 알앤비, 여성보컬,..."
5981,232355,"[봄을 담은 설렘 가득한 달달한 음악, 인디, 기분전환이 필요할때 듣고 싶은 음악들..."
476535,548777,"[하하하하하, 봄캐롤, 드라이브]"


> # 장르와 세부장르 입히기

In [63]:
# meta_dict[245872]
song_gn_dtl_gnr_basket_ll = []
issue_date_ll = []
album_name_ll = []
album_id_ll = []
artist_id_basket_ll = []
song_name_ll = []
song_gn_gnr_basket_ll = []
artist_name_basket_ll = []
for song in tqdm(song_id_ll):
    # print(song)
    # print(meta_dict[song])
    info = meta_dict[song]
    song_gn_dtl_gnr_basket = info['song_gn_dtl_gnr_basket']
    issue_date = info['issue_date']
    album_name = info['album_name']
    album_id = info['album_id']
    artist_id_basket = info['artist_id_basket']
    song_name = info['song_name']
    song_gn_gnr_basket = info['song_gn_gnr_basket']
    artist_name_basket = info['artist_name_basket']

    song_gn_dtl_gnr_basket_ll.append(song_gn_dtl_gnr_basket)
    issue_date_ll.append(issue_date)
    album_name_ll.append(album_name)
    album_id_ll.append(album_id)
    artist_id_basket_ll.append(artist_id_basket)
    song_name_ll.append(song_name)
    song_gn_gnr_basket_ll.append(song_gn_gnr_basket)
    artist_name_basket_ll.append(artist_name_basket)


100%|██████████| 615142/615142 [00:03<00:00, 179017.91it/s]


In [64]:
song2tag_df['song_gn_dtl_gnr_basket'] = song_gn_dtl_gnr_basket_ll
song2tag_df['issue_date'] = issue_date_ll
song2tag_df['album_name'] = album_name_ll
song2tag_df['album_id'] = album_id_ll
song2tag_df['artist_id_basket'] = artist_id_basket_ll
song2tag_df['song_name'] = song_name_ll
song2tag_df['song_gn_gnr_basket'] = song_gn_gnr_basket_ll
song2tag_df['artist_name_basket'] = artist_name_basket_ll

song2tag_df.sample(10)

,songID,tag,song_gn_dtl_gnr_basket,issue_date,album_name,album_id,artist_id_basket,song_name,song_gn_gnr_basket,artist_name_basket
539170,9526,"[잔잔한듯하지만 리듬타게 되는 국내 멜로우 힙합, 힙합, 잔잔한, 랩, 멜로우, H...","[GN0501, GN0304, GN0505, GN0301]",20170316,Exit,10046243,[1203540],Exit (Underwater) (Prod. Hassan Malik),"[GN0500, GN0300]",[Vincent Laurent]
185173,349755,"[마이매드팻다이어리, 락, haya 모던락&브릿, 락, 모던락 롤 하면서 듣는, 기...","[GN1002, GN1013, GN1001]",20090630,Festival Generation - Night,598252,[104369],Wonderwall,[GN1000],[Oasis]
244401,151749,"[카페에서 독서하기 좋은날♪, 힐링, 휴식, 설렘, 사랑]","[GN1912, GN1902, GN1901]",20060802,Yumekui (꿈을 먹는 벌레),329710,[161289],Tears,[GN1900],[Otsuka Ai]
115800,536976,"[운전할때 트는 음악(장르혼합), 드라이브, 운동, FAVORITE POPSONG!...",[GN0901],20141117,Four,2291064,[581612],Fool`s Gold,[GN0900],[One Direction]
291973,238436,"[<국내>내가 좋아하는 비트 메이커 20인 CHAPTER4-HIPHOP, 힙합, G...","[GN1501, GN0301, GN0302, GN1504]",20190830,"BEEHIVE (Feat. The Quiett, ZENE THE ZILLA, Dbo...",10322560,[169115],"BEEHIVE (Feat. The Quiett, ZENE THE ZILLA, Dbo...","[GN1500, GN0300]",[진보]
391218,87914,"[행복한 하루를 위한 피아노 모음, 행복, 힐링, 뉴에이지]","[GN1807, GN1801]",20150521,음악이 흐르는 나의 휴식 시간 (힐링 뉴에이지 피아노),208895,[750485],알람 소리,[GN1800],[몽당연필]
539277,598406,"[타임머신 타고 추억의 댄스 세계로, 90]",[GN0201],20070906,With A Twist,357673,[194724],부비 Party,[GN0200],[바나나 보트]
115483,617016,"[밤에 듣기편한 pop, 기분전환]",[GN1001],20121205,The A To Z Of Rock,2169160,[101893],The Final Countdown (Single Ver.),[GN1000],[Europe]
298869,188711,"[달콤달콤, 카페에서 듣고 싶은 100곡의 달달한 팝 음악, 달달한, 유정선배에게 ...",[GN0901],20151009,Acoustic Covers (어쿠스틱 커버: 어쿠스틱으로 듣는 리메이크 팝 히트곡),2643959,[100566],Don`t Dream It`s Over,[GN0900],[Sixpence None The Richer]
404847,80125,"[마음이 따듯해지는 크리스마스 Classical BGM, 겨울음악, 성가, 겨울노래...","[GN1601, GN1615, GN1613, GN1610]",20181018,GREAT SINGERS & MUSICIANS IN COPENHAGEN 1931-1939,10212921,[2727],Haydn : Die Schopfung Hob.XXI/2 Part.II - XXII...,[GN1600],[Various Artists]


> # csv로 빼기

In [65]:
song2tag_df.to_csv('song2tag_df.csv')